## Pseudo Labeling

In [2]:
import pandas as pd
import numpy as np
import cv2
import shutil
from glob import glob
import os
import matplotlib.pyplot as plt
from PIL import Image
import json
from tqdm import tqdm
from pycocotools.coco import COCO

In [3]:
# csv 파일 경로 수정
submission = pd.read_csv('/opt/ml/input/develop/submission/swinL_multiscale.csv')
data_dir = '/opt/ml/input/data'
train_json_path = '/opt/ml/input/data/new_train_all_anno_excluded.json'
pseudo_json_path = '/opt/ml/input/data/pseudo.json' # 저장될 경로
with open(train_json_path, 'r') as train_json:
    train_dict = json.load(train_json)

pseudo_dict = {}
pseudo_dict["info"] = train_dict['info']
pseudo_dict['licenses'] = train_dict['licenses']
pseudo_dict['images'] = train_dict['images']
pseudo_dict['annotations'] = train_dict['annotations']
pseudo_dict['categories'] = train_dict['categories']

img_dir = '/opt/ml/input/data/pseudo_mmseg/img_dir'
ann_dir = '/opt/ml/input/data/pseudo_mmseg/ann_dir'

In [ ]:
print(len(pseudo_dict['images']))
print(len(pseudo_dict['annotations']))

## Sol 2
.csv => 256x256 reshape => Image로 변환 후 jpg, png로 저장 후 mmseg에서 학습
기존 mmseg의 ann_dir, img_dir은 위의 img_dir, ann_dir 변수 대로 복사하였음.
그럼 kfold는 어떻게 하지?

In [ ]:
# start_img=0
# start_ann=0
# for i in tqdm(range(len(submission))):
#     cur = submission.iloc[i,:]
#     old_path = os.path.join(data_dir, cur['image_id'])
#     new_path = os.path.join(img_dir, str(start_img + i).zfill(4) + '.jpg')

#     shutil.copyfile(old_path, new_path)
    
#     arr = np.array(cur['PredictionString'].split()).reshape(256, 256)
#     mask = Image.fromarray(np.uint8(arr))
#     mask = mask.resize((512, 512), Image.LANCZOS)
#     plt.imshow(mask)
#     new_ann_path = os.path.join(ann_dir, str(start_ann + i).zfill(4) + '.png')
#     mask.save(new_ann_path)
#     break

In [4]:
print(submission.iloc[1])

image_id                                         batch_01_vt/0028.jpg
PredictionString    0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
Name: 1, dtype: object


## Sol 1
object detection 처럼 .csv를 coco format으로 저장

In [ ]:
start_img_id = train_dict['images'][-1]['id'] + 1
start_ann_id = train_dict['annotations'][-1]['id'] + 1

for i in tqdm(range(len(submission))):
    # submission 형태는 위의 print 참고
    now = submission.iloc[i] # i 번째 행 가져옴
    tmp_img_dict = {'width': 512, 'height': 512,
     'file_name': now['image_id'], 'license': 0,
      'flickr_url': None, 'coco_url': None, 
      'date_captured': '2021-01-06 09:09:58', 'id': start_img_id}
    
    pseudo_dict['images'].append(tmp_img_dict)

    # 256x256 arr에 prediction 저장
    arr = np.array(now['PredictionString'].split()).reshape(256,256)
    # 카테고리 별 좌표 저장할 변수
    category_arr = [[] for i in range(11)]

    # 512x512로 바꾸기 위해 arr을 Image로 바꾼 후
    # resize(512,512) 후 다시 int로 바꾸었음
    mask = Image.fromarray(np.uint8(arr))
    mask = mask.resize((512, 512), Image.LANCZOS)
    plt.imshow(mask)
    arr = np.uint8(mask)
    arr = np.array(arr, dtype='int')

    # category 별로 좌표 저장
    for x in range(512):
      for y in range(512):
        if 0 < arr[x][y] and arr[x][y] < 11:
          # 좌표니까 y, x로 저장하였음
          category_arr[arr[x][y]].append(y)
          category_arr[arr[x][y]].append(x)
    # 1~10 카테고리 탐색
    for n in range(1,11):
      # baseline이랑 똑같이 만드려고 category_arr[n]을 list로 감싸주었음
      tmp = []
      tmp.append(category_arr[n])
      # area, bbox, iscrowd는 아무거나
      tmp_ann_dict = {
        'image_id': start_img_id,
        'category_id': n,
        "segmentation": np.array(tmp, dtype=np.int8), 
        "area": 0, 
        "bbox": np.array([100,100,101,101], dtype=np.float32), 
        "iscrowd": 0,
        'id': start_ann_id}
      # segmentation이 존재해야 annotation에 저장
      if len(tmp_ann_dict['segmentation'][0]):
        pseudo_dict['annotations'].append(tmp_ann_dict)
        start_ann_id+=1

    start_img_id += 1

In [ ]:
print(len(pseudo_dict['images']))
print(len(pseudo_dict['annotations']))

In [ ]:
with open(pseudo_json_path, 'w', encoding='utf-8') as f:
    json.dump(pseudo_dict, f, ensure_ascii=False, indent=4)
